## Hello XTLib Store
This notebook serves as an introduction to the concepts and API's of XTLib.

Let's get started by importing some libraries that we will need for this notebook, including a few from XTLib.

In [1]:
import os
import sys

import time
import shutil
import tempfile
import numpy as np

# for xtlib internal development, we give preference to our working xtlib sources
sys.path.insert(0, os.getcwd() + "/..")

import xtlib
from xtlib.xt_store import XTStore

## XTLib Services
XTLib has 2 primary services:
    - store (where experiments and related files kept)
    - compute (used to scale up your ML runs)
    
In this notebook, we will focus on the **store** service.  T

The store that we access thru the service can be either file-based (local machine or server), or Azure-based 
(based on an Azure storage account).  Here, we will use a file-based store so we don't have to enter 
the Azure storage name and key. It will also enable a more predictable environment for our notebook.

**Note:** the store service API is the same, no matter if the store is file-based or Azure-based.

In [2]:
# create an azure store
# store = XTStore("myazurestore", "xxxxxxxxxxxxxx")

# create a file-based store
dir = "./temp_localstore_folder"

# ensure we start with a new store folder, so our notebook knows what to expect
if os.path.exists(dir):
    shutil.rmtree(dir)

# if dir doesn't exist, it will be created
store = XTStore(dir)

console.print(store)

## Store Concepts
 Within a store, you will find:
        - runs (a run is a single execution of your ML app on a machine)
        - experiments (an experiment is a named grouping of your runs)
        - workspaces (a workspace can hold multiple experiments and is also the unit of collaboration between users)
        - files (files can be associated with workspaces, experiments, and runs)

In [3]:
# let's start by enumerating the currently defined workspaces

ws_list = store.get_workspace_names()
console.print(ws_list)

[]


## As we might expect, there are not defined yet in our new store.
Let's try some more of the workspace functions

In [4]:
ws_name = "myws"

result = store.does_workspace_exist(ws_name)
console.print("result=", result)

if result:
    store.delete_workspace(ws_name)
    
store.create_workspace(ws_name)

result2 = store.does_workspace_exist(ws_name)
console.print("result2=", result2)


result= False
result2= True


## Experiments and Runs
There is currently no XTLib API for explictly managing experiment names (groups names for runs), but we can use experiment names as we create runs.  Let's see what that looks like

In [5]:
# create the run entity (think of this a logging the beginning of an ML training run)
exper_name = "my first experiment"
run_name = store.start_run(ws_name, exper_name)

run_name

'run1'

# OK, that seemed to work.
Typically, an ML app would log values of hyperparameters and metrics during the run.  Let's simulate an ML app
and do the logging, including the call to end_run().

In [6]:
# log hyperparameters used for this run
lr = .01
epochs = 30
store.log_run_event(ws_name, run_name, "hp", {"lr": lr, "epochs": epochs})
                                                     
# simulate some training                                                    
for epoch in range(epochs):
    time.sleep(.1)   
                                                      
    if epoch % 10 == 0:
                                                      
        console.print("epoch", epoch)
        val_loss = .5*np.random.random()
        val_acc = .5 + .5*np.random.random()
        
        # we can log hp/metrics one or multiple name/value pairs within a single call
        store.log_run_event(ws_name, run_name, "metrics", {"val_loss": val_loss})
        store.log_run_event(ws_name, run_name, "metrics", {"val_acc": val_acc})
        
# simulate end of training run
rollups = {"val_loss": val_loss, "val_acc": val_acc}
store.end_run(ws_name, run_name, status="completed", exit_code=0, metrics_rollup_dict=rollups)
                                                      
console.print("end of our run.")

epoch 0
epoch 10
epoch 20
end of our run.


## Now, we have a completed run entity
If we were using a tool like XT to launch our ML apps, it would capture certain files for us.
These can be grouped into BEFORE files (those that existed in our working directory before the run) and
AFTER files, a snapshot of the files in the working directory after the run completes.

## Let's upload these types of files to our store from our code


In [7]:
# create a source file
fn_source = "myapp.py"
with open(fn_source, "w") as tfile:
    tfile.write("import sys\nprint('hello world')\n")

# create a console output file
fn_console = "console.txt"
with open(fn_console, "w") as tfile:
    tfile.write("hello world\n")

# upload source file to BEFORE/AFTER dirs of run
store.upload_file_to_run(ws_name, run_name, "before/" + fn_source, fn_source)
store.upload_file_to_run(ws_name, run_name, "after/" + fn_source, fn_source)

# upload console file to AFTER dir of run
store.upload_file_to_run(ws_name, run_name, "after/" + fn_console, fn_console)


## Last step of Notebook
As a last stop, we recommend that you examine the file-based store that was created in this notebook.  This can be done using the Jupyter Console tab in your browser (where you located and opened this notebook) - look for the directory:

    - temp_localstore_folder
    
If you happened to change this to work with your Azure account, you can use Azure Storage Explorer or the Azure portal to examine the container created for your workspace and the blobs for the files within the workspace.
    
